# PyTorch
## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s below. <==_

In [ ]:
%pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1

## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s above. <==_

# Hugging Face Transformers
## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s below. <==_

In [ ]:
%pip install --disable-pip-version-check -q transformers==4.26.1 datasets==2.9.0 accelerate==0.17.0 bitsandbytes==0.37.0 promptsource==0.2.3

## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s above. <==_

# Ray
## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s below. <==_

In [3]:
%pip install --disable-pip-version-check -q ray==2.3.0

Note: you may need to restart the kernel to use updated packages.


## _==> Please ignore all WARNINGs and ERRORs from the `pip install`'s above. <==_

In [4]:
# from datasets import Dataset
# import ray

# lm_dataset_train = Dataset.from_parquet('../data/train/*.parquet')

# train_dataset = ray.data.from_huggingface(
#     lm_dataset_train
# )

In [ ]:
# We are only testing Causal Language Modeling here
model_checkpoint = "bigscience/bloomz-560m"

def train_function(train_dataset, eval_dataset=None, **config):
    # Check that train_dataset has len
#    assert len(train_dataset)

    model_config = AutoConfig.from_pretrained(model_checkpoint)
    model = AutoModelForCausalLM.from_config(model_config)
    # evaluation_strategy = (
    #     config.pop("evaluation_strategy", "epoch") if eval_dataset else "no"
    # )
    training_args = TrainingArguments(
        f"{model_checkpoint}-amazon-reviews",
#        evaluation_strategy=config.pop("evaluation_strategy", "epoch") if eval_dataset else "no",
#        logging_strategy=config.pop("logging_strategy", "epoch"),
        num_train_epochs=config.pop("epochs", 1),
        learning_rate=config.pop("learning_rate", 2e-5),
        weight_decay=0.01,
#        disable_tqdm=True,
        no_cuda=True,
#        save_strategy=config.pop("save_strategy", "epoch"),
        **config,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
#        eval_dataset=eval_dataset,
    )
    return trainer

In [16]:
import pandas as pd
import pytest
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

import ray
ray.shutdown()

import ray.data
from ray.exceptions import RayTaskError
from ray.train.huggingface import (
    HuggingFaceTrainer,
    HuggingFaceCheckpoint,
)
from ray.air.config import ScalingConfig

In [17]:
import multiprocessing

num_cpus = multiprocessing.cpu_count()

def ray_start():
    address_info = ray.init(num_workers=num_cpus-20)
    yield address_info
    # The code after the yield will run as teardown code.
    ray.shutdown()

In [27]:
ray.shutdown()
ray_start()

# # 16 first rows of tokenized wikitext-2-raw-v1 training & validation
train_data = """
{"input_ids":{"0":[238,437,222,75,89,337,65,425,716,229,593,266,41,41,238,252,241,213,74,127,173,826,437,222,75,89,337,65,428,466,609,202,67,412,204,425,716,229,593,311,329,422,220,216,69,466,159,151,168,99,150,192,112,148,161,106,148,163,112,148,162,94,148,163,104,148,162,193,148,163,98,148,163,103,148,162,95,19,201,250,211,209,437,222,75,89,337,65,218,198,265,668,234,70,888,428,310,201,597,200,264,237,652,401,227,281,437,222,75,89,337,65,425,716,229,593,266,41,41,573,83,446,329,422,220,201,301,196],"1":[194,538,452,494,234,277,535,223,348,446,79,674,934,204,307,742,71,65,225,259,204,427,14,54,719,274,198,290,76,288,51,84,291,290,349,614,209,789,234,620,221,329,220,85,430,546,17,221,329,422,220,201,322,301,198,244,825,674,221,198,437,222,75,89,337,65,843,209,338,77,520,622,223,198,203,386,214,283,233,218,194,538,452,225,237,222,277,582,674,520,288,281,457,224,401,262,382,621,201,198,285,521,358,302,83,224,217,222,234,76,227,198,455,674,225,774,83,198,239,318,251,248,382,239,201,196,224,213],"2":[222,230,243,211,430,434,211,553,86,223,198,254,291,218,315,366,427,661,198,241,262,497,338,85,635,220,312,217,516,889,644,202,84,841,394,205,473,524,225,390,224,211,744,872,198,266,77,473,453,434,211,239,247,222,251,208,89,297,336,213,239,209,252,261,674,959,934,408,221,546,16,201,212,217,488,223,547,196,768,411,224,349,233,218,198,630,231,588,263,437,222,75,89,337,65,425,716,229,593,266,41,209,980,515,322,751,890,198,285,325,364,641,208,895,218,198,843,201,322,471,699,87,242,230,512,414,234,486],"3":[74,431,967,201,781,281,230,368,223,198,674,604,274,71,272,223,274,843,664,67,249,320,209,425,217,257,280,541,474,199,297,65,211,65,282,200,74,235,225,441,316,199,282,211,316,72,73,241,368,273,279,79,857,751,564,204,340,577,86,632,821,337,216,201,308,362,296,437,222,75,89,337,65,425,716,229,593,266,41,970,210,236,368,216,72,73,339,90,655,65,209,240,768,411,927,218,730,320,232,325,914,198,518,337,381,209,261,674,268,83,919,213,223,642,69,258,203,302,71,307,259,288,268,78,209,252,530,922],"4":[296,729,211,388,203,222,216,221,329,422,220,201,225,258,224,314,838,307,857,329,422,220,216,69,225,206,216,612,987,790,209,929,734,626,201,322,891,777,231,469,76,79,256,614,449,242,201,308,362,296,326,875,325,204,159,204,522,221,318,402,576,218,295,544,209,530,258,471,486,978,204,587,537,71,65,225,326,946,222,348,446,79,326,273,291,843,209,300,467,227,250,284,203,222,216,218,437,222,75,89,337,65,425,716,229,593,266,41,201,437,222,75,89,337,65,425,716,229,593,266,41,41,258,410,844,222,968,201],"5":[445,196,214,220,877,76,291,441,208,551,234,296,198,674,268,83,875,325,204,159,204,522,258,734,620,221,546,20,209,259,204,427,14,54,719,602,751,564,227,198,214,476,309,854,296,198,934,408,218,437,222,75,89,337,65,466,240,90,447,789,86,270,286,233,274,198,290,76,288,51,84,291,493,209,252,238,238,315,386,520,288,238,238,252,662,296,577,86,632,437,222,75,89,275,65,425,716,229,593,267,728,201,437,222,75,89,337,65,425,716,229,593,266,41,41,301,196,194,538,452,494,234,277,535,223,674,756,535],"6":[320,194,657,449,228,76,218,196,230,243,211,430,434,211,225,194,657,540,221,230,638,443,872,554,303,89,274,613,209,403,210,346,390,227,352,718,341,229,219,680,277,250,783,224,220,979,296,326,273,380,878,280,224,349,314,868,201,296,878,690,405,69,368,223,540,73,417,718,348,79,229,204,405,69,262,72,219,463,66,593,225,540,73,417,718,434,86,79,229,204,601,88,84,209,261,535,199,344,71,566,216,718,196,843,218,250,197,383,230,638,443,201,267,82,256,957,434,76,722,204,281,230,422,83,295,684,293,214],"7":[202,539,518,786,204,718,225,237,520,288,204,281,543,390,434,76,722,204,209,261,494,933,227,253,532,285,521,844,291,263,198,230,422,348,217,346,897,483,223,263,326,785,272,271,779,535,199,268,83,528,228,532,466,646,496,205,381,233,301,359,791,204,201,198,548,301,359,222,204,555,227,198,535,199,209,339,286,83,446,230,638,443,201,198,535,199,878,690,237,246,221,196,212,600,201,756,434,868,684,293,212,431,249,968,225,878,280,267,859,299,205,438,260,83,209,517,362,83,446,198,906,285,521,230,638,443,390,878],"8":[280,277,753,778,747,230,638,443,691,761,227,962,652,242,203,387,256,230,974,320,209,929,198,674,268,83,981,84,233,201,882,522,222,866,335,216,390,434,76,722,204,201,766,218,642,232,336,223,196,232,278,294,962,70,229,512,89,702,244,607,824,221,198,237,246,218,198,674,209,261,202,390,471,250,402,69,203,273,306,291,253,234,967,691,380,227,198,674,268,83,506,906,331,228,828,201,308,910,543,194,657,196,723,89,931,210,494,234,209,252,261,674,268,83,219,668,234,809,246,303,201,198,265,76,73,52,58,809],"9":[246,303,201,301,212,217,337,204,547,970,264,340,437,222,75,89,275,65,425,716,229,593,209,300,519,230,638,443,201,535,320,359,791,253,532,434,211,487,223,196,227,80,277,231,469,224,320,80,395,388,218,198,219,668,234,70,888,230,422,466,263,298,196,878,280,301,359,791,204,201,198,535,199,758,216,198,878,280,196,590,198,219,668,234,70,888,221,244,825,277,224,320,200,209,240,878,280,684,669,733,263,298,503,277,194,564,201,445,878,690,684,293,267,82,373,204,230,512,414,234,194,564,83,319,198,875,213,399],"10":[218,548,878,690,268,194,564,83,209,338,532,878,280,499,196,214,888,225,231,333,567,218,758,915,250,273,616,307,444,240,557,315,971,411,209,397,80,227,254,415,878,690,684,293,741,474,204,227,196,887,234,230,638,233,209,300,519,674,520,288,201,878,690,206,351,798,573,159,347,203,249,269,72,223,232,65,354,579,227,642,201,781,281,444,331,222,299,634,594,83,311,282,48,310,267,269,84,223,250,284,371,812,656,722,204,573,307,554,303,89,569,394,83,209,338,532,878,280,499,753,778,239,290,279,242,453,83,239],"11":[201,203,75,351,83,434,73,387,69,227,253,532,878,280,209,261,89,390,231,272,271,204,587,239,290,320,200,222,290,279,242,453,239,201,400,390,221,78,365,203,75,351,83,295,648,334,434,222,280,204,434,76,382,548,87,854,231,640,380,307,198,285,521,225,684,253,211,294,331,76,80,371,556,80,204,69,196,878,280,201,225,239,265,668,234,290,279,242,453,83,239,201,400,390,267,228,574,718,462,198,674,225,308,683,83,267,82,373,219,79,675,227,196,878,280,209,236,79,409,217,78,265,668,234,290,279,242,453],"12":[83,201,253,532,878,280,499,196,434,73,387,69,239,259,226,690,236,614,239,201,196,267,82,271,277,219,620,203,75,351,194,614,295,684,293,788,227,772,387,482,225,250,197,75,962,652,242,203,75,351,83,209,425,217,257,690,471,490,241,80,262,453,240,66,243,839,295,267,82,373,642,194,303,80,210,430,219,79,419,83,263,198,219,668,234,70,888,466,384,260,84,684,733,272,365,239,300,860,703,77,325,239,225,758,69,196,590,198,219,668,234,70,888,296,462,897,234,84,223,342,240,557,290,79,594,267,971,411],"13":[201,198,878,280,789,243,65,684,392,347,84,587,468,239,437,222,75,89,337,65,304,477,239,225,527,479,896,197,67,551,234,201,732,266,77,67,65,684,194,217,71,269,230,512,414,234,554,303,89,434,868,296,468,331,336,89,615,422,200,209,252,236,228,343,83,390,231,272,271,204,587,214,388,472,481,216,466,852,462,83,201,808,722,84,228,343,320,201,811,415,320,201,317,220,67,320,225,704,77,356,68,241,636,853,209,236,228,343,320,684,203,87,211,309,472,481,216,307,370,568,223,444,741,474,204,615,422,200],"14":[209,425,568,223,472,481,804,216,410,267,682,264,196,465,395,198,285,208,83,267,890,732,221,196,577,86,632,472,481,209,312,287,348,640,521,221,219,668,234,201,377,473,73,507,634,594,83,390,196,603,204,227,198,203,387,256,201,400,390,231,333,608,286,204,587,214,388,962,652,242,569,608,286,216,392,944,307,198,821,482,203,387,256,201,196,641,208,447,962,652,223,340,989,267,728,268,922,72,335,218,231,333,608,286,223,227,962,652,242,434,211,194,998,216,209,252,238,238,290,76,279,238,238,252,261,674,194,368],"15":[216,385,571,661,198,241,262,497,338,85,635,220,312,217,209,315,366,406,704,77,89,241,387,256,493,18,18,201,471,876,281,239,261,318,251,248,382,239,201,390,196,224,213,222,230,243,211,430,434,211,441,316,204,218,212,82,273,197,637,201,214,356,474,541,199,690,201,225,230,243,211,430,555,483,320,305,607,237,222,254,728,390,159,199,620,340,198,714,83,225,817,200,555,229,73,417,237,652,401,227,307,725,66,320,209,339,82,68,665,307,198,315,366,406,230,243,211,430,227,889,198,659,231,568,199,435,230,638]},"attention_mask":{"0":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"1":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"2":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"3":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"4":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"5":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"6":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"7":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"8":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"9":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"10":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"11":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"12":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"13":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"14":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"15":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]},"labels":{"0":[238,437,222,75,89,337,65,425,716,229,593,266,41,41,238,252,241,213,74,127,173,826,437,222,75,89,337,65,428,466,609,202,67,412,204,425,716,229,593,311,329,422,220,216,69,466,159,151,168,99,150,192,112,148,161,106,148,163,112,148,162,94,148,163,104,148,162,193,148,163,98,148,163,103,148,162,95,19,201,250,211,209,437,222,75,89,337,65,218,198,265,668,234,70,888,428,310,201,597,200,264,237,652,401,227,281,437,222,75,89,337,65,425,716,229,593,266,41,41,573,83,446,329,422,220,201,301,196],"1":[194,538,452,494,234,277,535,223,348,446,79,674,934,204,307,742,71,65,225,259,204,427,14,54,719,274,198,290,76,288,51,84,291,290,349,614,209,789,234,620,221,329,220,85,430,546,17,221,329,422,220,201,322,301,198,244,825,674,221,198,437,222,75,89,337,65,843,209,338,77,520,622,223,198,203,386,214,283,233,218,194,538,452,225,237,222,277,582,674,520,288,281,457,224,401,262,382,621,201,198,285,521,358,302,83,224,217,222,234,76,227,198,455,674,225,774,83,198,239,318,251,248,382,239,201,196,224,213],"2":[222,230,243,211,430,434,211,553,86,223,198,254,291,218,315,366,427,661,198,241,262,497,338,85,635,220,312,217,516,889,644,202,84,841,394,205,473,524,225,390,224,211,744,872,198,266,77,473,453,434,211,239,247,222,251,208,89,297,336,213,239,209,252,261,674,959,934,408,221,546,16,201,212,217,488,223,547,196,768,411,224,349,233,218,198,630,231,588,263,437,222,75,89,337,65,425,716,229,593,266,41,209,980,515,322,751,890,198,285,325,364,641,208,895,218,198,843,201,322,471,699,87,242,230,512,414,234,486],"3":[74,431,967,201,781,281,230,368,223,198,674,604,274,71,272,223,274,843,664,67,249,320,209,425,217,257,280,541,474,199,297,65,211,65,282,200,74,235,225,441,316,199,282,211,316,72,73,241,368,273,279,79,857,751,564,204,340,577,86,632,821,337,216,201,308,362,296,437,222,75,89,337,65,425,716,229,593,266,41,970,210,236,368,216,72,73,339,90,655,65,209,240,768,411,927,218,730,320,232,325,914,198,518,337,381,209,261,674,268,83,919,213,223,642,69,258,203,302,71,307,259,288,268,78,209,252,530,922],"4":[296,729,211,388,203,222,216,221,329,422,220,201,225,258,224,314,838,307,857,329,422,220,216,69,225,206,216,612,987,790,209,929,734,626,201,322,891,777,231,469,76,79,256,614,449,242,201,308,362,296,326,875,325,204,159,204,522,221,318,402,576,218,295,544,209,530,258,471,486,978,204,587,537,71,65,225,326,946,222,348,446,79,326,273,291,843,209,300,467,227,250,284,203,222,216,218,437,222,75,89,337,65,425,716,229,593,266,41,201,437,222,75,89,337,65,425,716,229,593,266,41,41,258,410,844,222,968,201],"5":[445,196,214,220,877,76,291,441,208,551,234,296,198,674,268,83,875,325,204,159,204,522,258,734,620,221,546,20,209,259,204,427,14,54,719,602,751,564,227,198,214,476,309,854,296,198,934,408,218,437,222,75,89,337,65,466,240,90,447,789,86,270,286,233,274,198,290,76,288,51,84,291,493,209,252,238,238,315,386,520,288,238,238,252,662,296,577,86,632,437,222,75,89,275,65,425,716,229,593,267,728,201,437,222,75,89,337,65,425,716,229,593,266,41,41,301,196,194,538,452,494,234,277,535,223,674,756,535],"6":[320,194,657,449,228,76,218,196,230,243,211,430,434,211,225,194,657,540,221,230,638,443,872,554,303,89,274,613,209,403,210,346,390,227,352,718,341,229,219,680,277,250,783,224,220,979,296,326,273,380,878,280,224,349,314,868,201,296,878,690,405,69,368,223,540,73,417,718,348,79,229,204,405,69,262,72,219,463,66,593,225,540,73,417,718,434,86,79,229,204,601,88,84,209,261,535,199,344,71,566,216,718,196,843,218,250,197,383,230,638,443,201,267,82,256,957,434,76,722,204,281,230,422,83,295,684,293,214],"7":[202,539,518,786,204,718,225,237,520,288,204,281,543,390,434,76,722,204,209,261,494,933,227,253,532,285,521,844,291,263,198,230,422,348,217,346,897,483,223,263,326,785,272,271,779,535,199,268,83,528,228,532,466,646,496,205,381,233,301,359,791,204,201,198,548,301,359,222,204,555,227,198,535,199,209,339,286,83,446,230,638,443,201,198,535,199,878,690,237,246,221,196,212,600,201,756,434,868,684,293,212,431,249,968,225,878,280,267,859,299,205,438,260,83,209,517,362,83,446,198,906,285,521,230,638,443,390,878],"8":[280,277,753,778,747,230,638,443,691,761,227,962,652,242,203,387,256,230,974,320,209,929,198,674,268,83,981,84,233,201,882,522,222,866,335,216,390,434,76,722,204,201,766,218,642,232,336,223,196,232,278,294,962,70,229,512,89,702,244,607,824,221,198,237,246,218,198,674,209,261,202,390,471,250,402,69,203,273,306,291,253,234,967,691,380,227,198,674,268,83,506,906,331,228,828,201,308,910,543,194,657,196,723,89,931,210,494,234,209,252,261,674,268,83,219,668,234,809,246,303,201,198,265,76,73,52,58,809],"9":[246,303,201,301,212,217,337,204,547,970,264,340,437,222,75,89,275,65,425,716,229,593,209,300,519,230,638,443,201,535,320,359,791,253,532,434,211,487,223,196,227,80,277,231,469,224,320,80,395,388,218,198,219,668,234,70,888,230,422,466,263,298,196,878,280,301,359,791,204,201,198,535,199,758,216,198,878,280,196,590,198,219,668,234,70,888,221,244,825,277,224,320,200,209,240,878,280,684,669,733,263,298,503,277,194,564,201,445,878,690,684,293,267,82,373,204,230,512,414,234,194,564,83,319,198,875,213,399],"10":[218,548,878,690,268,194,564,83,209,338,532,878,280,499,196,214,888,225,231,333,567,218,758,915,250,273,616,307,444,240,557,315,971,411,209,397,80,227,254,415,878,690,684,293,741,474,204,227,196,887,234,230,638,233,209,300,519,674,520,288,201,878,690,206,351,798,573,159,347,203,249,269,72,223,232,65,354,579,227,642,201,781,281,444,331,222,299,634,594,83,311,282,48,310,267,269,84,223,250,284,371,812,656,722,204,573,307,554,303,89,569,394,83,209,338,532,878,280,499,753,778,239,290,279,242,453,83,239],"11":[201,203,75,351,83,434,73,387,69,227,253,532,878,280,209,261,89,390,231,272,271,204,587,239,290,320,200,222,290,279,242,453,239,201,400,390,221,78,365,203,75,351,83,295,648,334,434,222,280,204,434,76,382,548,87,854,231,640,380,307,198,285,521,225,684,253,211,294,331,76,80,371,556,80,204,69,196,878,280,201,225,239,265,668,234,290,279,242,453,83,239,201,400,390,267,228,574,718,462,198,674,225,308,683,83,267,82,373,219,79,675,227,196,878,280,209,236,79,409,217,78,265,668,234,290,279,242,453],"12":[83,201,253,532,878,280,499,196,434,73,387,69,239,259,226,690,236,614,239,201,196,267,82,271,277,219,620,203,75,351,194,614,295,684,293,788,227,772,387,482,225,250,197,75,962,652,242,203,75,351,83,209,425,217,257,690,471,490,241,80,262,453,240,66,243,839,295,267,82,373,642,194,303,80,210,430,219,79,419,83,263,198,219,668,234,70,888,466,384,260,84,684,733,272,365,239,300,860,703,77,325,239,225,758,69,196,590,198,219,668,234,70,888,296,462,897,234,84,223,342,240,557,290,79,594,267,971,411],"13":[201,198,878,280,789,243,65,684,392,347,84,587,468,239,437,222,75,89,337,65,304,477,239,225,527,479,896,197,67,551,234,201,732,266,77,67,65,684,194,217,71,269,230,512,414,234,554,303,89,434,868,296,468,331,336,89,615,422,200,209,252,236,228,343,83,390,231,272,271,204,587,214,388,472,481,216,466,852,462,83,201,808,722,84,228,343,320,201,811,415,320,201,317,220,67,320,225,704,77,356,68,241,636,853,209,236,228,343,320,684,203,87,211,309,472,481,216,307,370,568,223,444,741,474,204,615,422,200],"14":[209,425,568,223,472,481,804,216,410,267,682,264,196,465,395,198,285,208,83,267,890,732,221,196,577,86,632,472,481,209,312,287,348,640,521,221,219,668,234,201,377,473,73,507,634,594,83,390,196,603,204,227,198,203,387,256,201,400,390,231,333,608,286,204,587,214,388,962,652,242,569,608,286,216,392,944,307,198,821,482,203,387,256,201,196,641,208,447,962,652,223,340,989,267,728,268,922,72,335,218,231,333,608,286,223,227,962,652,242,434,211,194,998,216,209,252,238,238,290,76,279,238,238,252,261,674,194,368],"15":[216,385,571,661,198,241,262,497,338,85,635,220,312,217,209,315,366,406,704,77,89,241,387,256,493,18,18,201,471,876,281,239,261,318,251,248,382,239,201,390,196,224,213,222,230,243,211,430,434,211,441,316,204,218,212,82,273,197,637,201,214,356,474,541,199,690,201,225,230,243,211,430,555,483,320,305,607,237,222,254,728,390,159,199,620,340,198,714,83,225,817,200,555,229,73,417,237,652,401,227,307,725,66,320,209,339,82,68,665,307,198,315,366,406,230,243,211,430,227,889,198,659,231,568,199,435,230,638]}}
"""

validation_data = """
{"input_ids":{"0":[238,282,249,217,283,267,251,77,217,283,238,252,282,249,217,283,267,251,77,217,283,201,876,281,198,338,85,635,69,220,250,706,246,199,371,597,200,250,706,246,199,201,301,196,753,346,218,472,655,204,250,706,246,199,340,198,253,226,612,775,76,373,229,339,298,220,201,259,204,211,199,476,69,220,742,65,225,540,83,218,198,265,76,394,742,65,209,530,301,472,316,539,691,380,227,198,764,220,250,706,246,199,201,282,14,196,760,220,283,209,530,977,267,859,227,196,250,213,71,299,218,605,16,212,77,311,289,20],"1":[221,310,225,196,230,481,218,605,393,243,432,880,83,311,245,19,250,66,310,201,225,293,819,196,660,80,229,85,435,224,899,218,472,655,83,209,357,250,822,201,198,250,706,246,320,390,841,467,201,669,527,249,223,239,250,706,246,199,237,68,239,263,212,680,223,209,259,761,205,438,260,83,221,198,203,327,495,201,731,67,223,253,71,71,83,400,390,212,217,337,204,307,198,214,303,222,216,274,578,227,196,544,806,232,208,309,223,587,385,858,851,229,768,86,65,69,209,282,249,217,283,267,251,77,217,283,301,196],"2":[823,264,928,69,303,204,214,623,201,225,301,206,271,539,212,971,378,84,487,223,250,706,246,199,224,279,83,201,659,264,196,590,198,863,424,984,593,209,252,238,238,300,216,67,337,381,233,238,238,252,282,249,217,283,267,251,77,217,283,301,196,768,411,212,82,431,571,220,201,296,196,219,335,89,250,213,71,299,578,227,605,16,624,273,269,345,311,289,20,221,310,225,615,363,223,578,227,502,459,605,393,243,432,880,83,311,920,459,245,19,250,66,310,201,308,910,198,250,706,246,320,212,971,378,84,221,250,706],"3":[246,199,224,279,83,390,487,957,289,19,459,428,24,212,77,311,670,459,916,221,310,908,225,615,363,796,511,629,459,289,511,289,393,71,311,245,511,502,459,493,511,670,250,66,310,209,317,783,548,212,82,431,571,738,201,250,706,246,320,490,196,232,364,377,316,559,234,851,400,543,230,431,203,458,221,371,975,227,267,859,201,221,196,344,67,382,994,253,67,68,89,83,215,311,230,235,76,84,223,310,209,795,977,205,438,260,951,194,273,216,196,544,274,407,426,71,250,706,246,320,201,445,671,202,226,216,227,263],"4":[298,253,292,89,245,459,289,818,274,768,71,199,326,273,637,209,252,261,455,224,899,218,224,350,73,343,335,83,301,454,77,204,296,196,768,411,201,281,89,77,77,269,82,452,224,899,218,472,655,83,209,261,768,71,199,496,301,198,239,212,82,283,294,239,201,225,499,159,590,204,254,335,306,216,788,274,212,82,283,72,223,577,89,440,198,548,301,198,239,212,286,280,239,201,400,499,392,217,80,221,78,199,159,204,71,216,201,225,301,788,274,232,636,223,371,194,383,223,198,577,89,209,397,83,957,201,198,409],"5":[678,472,655,301,198,212,82,283,294,201,225,198,358,420,301,198,212,286,280,209,252,261,377,316,559,234,851,301,267,757,417,841,467,485,402,69,201,296,405,279,83,295,767,222,216,298,201,225,407,416,284,993,284,209,261,237,68,713,418,741,375,73,380,296,250,706,246,320,669,528,383,83,550,212,680,223,209,795,205,438,260,83,527,848,201,221,250,822,201,198,237,68,224,278,408,196,246,65,88,220,299,197,301,219,549,227,196,344,563,197,981,88,201,445,198,981,88,301,219,228,75,213,578,307,198,331,208,218],"6":[212,680,223,201,734,226,223,198,237,68,224,278,408,209,252,261,472,316,328,691,941,218,282,14,267,251,77,217,283,301,198,764,220,250,706,246,199,201,282,249,217,283,196,760,220,283,209,261,506,753,346,390,723,89,203,273,243,217,201,225,684,293,212,973,204,982,778,73,417,201,308,910,232,89,66,82,271,83,390,434,76,965,539,227,205,438,260,221,198,206,627,203,856,444,358,568,216,804,410,547,76,422,209,261,506,753,346,684,293,231,333,223,85,736,307,196,845,218,878,280,333,790,466,252,261,494,246,82],"7":[327,218,282,14,196,760,220,283,293,819,496,371,604,405,828,263,198,423,207,320,446,201,400,390,250,394,223,221,282,14,267,251,77,217,283,209,252,261,405,828,263,198,472,655,83,218,282,14,196,760,220,283,390,237,68,371,237,68,277,194,73,354,204,201,732,244,607,218,282,14,267,251,77,217,283,390,305,560,371,305,560,277,194,73,354,204,209,252,261,423,207,320,446,218,198,472,655,218,282,14,196,760,220,283,301,371,220,411,371,237,68,201,732,295,218,282,14,267,251,77,217,283,301,942,251,89,305,560],"8":[371,723,89,224,842,237,68,209,252,238,238,317,822,212,89,67,234,238,238,252,304,303,842,282,14,267,251,77,217,283,237,532,359,88,779,230,208,260,379,646,543,490,267,228,574,227,196,212,217,422,571,250,213,71,299,218,672,16,459,672,21,955,273,269,345,311,428,511,245,459,428,511,428,221,310,201,756,226,230,222,216,230,208,447,319,196,203,76,420,264,947,366,199,203,529,69,209,259,761,194,998,229,417,205,438,260,83,221,203,327,495,708,196,489,242,264,230,235,76,744,214,303,842,201,305,607,598,800],"9":[301,817,717,749,678,201,225,196,232,364,277,598,800,204,230,842,209,261,214,303,842,212,217,337,216,198,253,71,71,83,274,578,227,837,904,299,83,201,897,483,223,263,198,194,303,473,208,447,201,569,257,458,227,468,224,234,343,335,83,209,304,303,222,216,212,217,488,223,253,71,71,83,390,203,810,227,293,239,219,199,337,204,239,225,684,293,824,718,462,198,544,209,252,261,253,71,71,83,232,208,309,319,254,420,201,225,198,768,86,65,69,203,87,273,227,198,206,475,833,70,571,756,543,231,337,678,296,198],"10":[205,298,220,212,260,592,745,201,577,89,223,263,159,90,79,343,76,858,851,209,795,285,491,896,270,86,216,697,230,235,76,745,225,250,404,83,274,916,459,428,21,792,83,209,929,198,244,825,230,235,76,84,201,198,536,85,86,213,515,194,368,216,263,196,740,472,316,199,227,198,486,512,201,225,486,343,745,196,219,242,72,229,250,347,328,89,234,209,261,536,85,86,213,243,216,390,765,202,264,359,213,221,198,206,627,201,225,390,634,210,264,876,201,308,910,543,390,876,227,293,212,422,614,218,231,278,71,223],"11":[972,579,388,219,260,859,83,209,530,301,928,273,380,295,669,245,768,86,65,221,253,292,89,464,562,802,833,86,272,216,227,198,219,242,72,229,224,72,626,209,312,787,543,237,532,196,212,217,422,571,250,213,71,299,218,916,230,77,311,796,511,502,25,221,310,201,198,536,85,86,213,243,216,409,429,444,219,260,859,83,225,285,367,444,486,512,250,272,216,209,252,238,238,300,333,608,286,233,238,238,252,282,249,217,283,267,251,77,217,283,301,824,772,973,198,254,663,277,253,226,612,775,76,373,229,339,298,220],"12":[340,254,349,294,78,318,210,683,227,198,240,90,79,345,225,259,210,375,67,79,201,410,867,198,265,222,84,229,742,65,209,530,301,471,846,242,221,659,218,198,259,204,211,199,476,69,220,742,65,201,669,230,638,223,340,198,203,395,233,253,404,218,247,202,563,201,225,308,362,669,198,254,663,277,206,328,767,404,218,198,265,76,394,742,65,209,261,254,349,294,78,77,419,883,306,524,390,824,221,198,318,210,591,71,406,214,74,412,83,236,89,83,70,74,412,213,225,318,412,70,636,65,201,221,83,446,198,704],"13":[321,229,247,275,67,234,209,252,261,753,346,684,293,231,272,271,204,587,784,267,213,269,229,417,231,333,197,321,883,306,524,201,496,206,271,216,80,202,256,883,306,291,201,225,697,400,490,231,829,71,204,231,467,227,947,366,253,465,395,388,883,306,291,203,529,216,201,729,83,551,264,231,467,227,486,978,291,227,198,844,222,554,86,275,200,408,209,261,455,218,198,399,301,198,883,306,291,218,250,706,246,320,340,254,349,294,78,318,210,683,201,400,490,504,237,652,401,227,281,198,239,230,271,78,420,277,203,302],"14":[250,706,246,199,239,209,261,883,306,524,221,198,259,204,211,199,476,69,220,742,65,390,231,333,197,321,340,244,607,221,198,775,76,373,229,339,298,220,209,261,250,404,231,333,197,321,883,306,291,301,824,221,198,318,269,294,508,83,466,203,251,520,216,340,198,339,419,320,67,195,352,69,360,231,333,197,321,340,244,607,713,791,204,221,198,318,663,742,65,371,811,76,424,425,786,248,209,252,775,84,303,381,83,490,504,746,227,450,228,501,298,282,14,267,251,77,217,283,227,752,903,69,222,325,201,308,362,83],"15":[446,548,338,85,635,69,220,753,346,781,281,198,159,204,551,234,212,314,66,201,247,220,67,199,224,330,260,283,209,265,269,687,323,16,20,225,323,17,20,201,496,955,233,250,706,246,199,768,86,65,69,360,734,620,340,232,208,67,294,346,221,300,302,204,197,201,445,198,753,346,992,410,527,479,928,372,76,736,817,209,252,238,238,338,67,727,89,238,238,252,240,68,512,282,14,267,251,77,217,283,250,388,263,198,963,242,222,598,76,70,319,897,299,83,218,796,459,916,16,922,345,311,796,459,493,25,18,214]},"attention_mask":{"0":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"1":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"2":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"3":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"4":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"5":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"6":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"7":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"8":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"9":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"10":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"11":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"12":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"13":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"14":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],"15":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]},"labels":{"0":[238,282,249,217,283,267,251,77,217,283,238,252,282,249,217,283,267,251,77,217,283,201,876,281,198,338,85,635,69,220,250,706,246,199,371,597,200,250,706,246,199,201,301,196,753,346,218,472,655,204,250,706,246,199,340,198,253,226,612,775,76,373,229,339,298,220,201,259,204,211,199,476,69,220,742,65,225,540,83,218,198,265,76,394,742,65,209,530,301,472,316,539,691,380,227,198,764,220,250,706,246,199,201,282,14,196,760,220,283,209,530,977,267,859,227,196,250,213,71,299,218,605,16,212,77,311,289,20],"1":[221,310,225,196,230,481,218,605,393,243,432,880,83,311,245,19,250,66,310,201,225,293,819,196,660,80,229,85,435,224,899,218,472,655,83,209,357,250,822,201,198,250,706,246,320,390,841,467,201,669,527,249,223,239,250,706,246,199,237,68,239,263,212,680,223,209,259,761,205,438,260,83,221,198,203,327,495,201,731,67,223,253,71,71,83,400,390,212,217,337,204,307,198,214,303,222,216,274,578,227,196,544,806,232,208,309,223,587,385,858,851,229,768,86,65,69,209,282,249,217,283,267,251,77,217,283,301,196],"2":[823,264,928,69,303,204,214,623,201,225,301,206,271,539,212,971,378,84,487,223,250,706,246,199,224,279,83,201,659,264,196,590,198,863,424,984,593,209,252,238,238,300,216,67,337,381,233,238,238,252,282,249,217,283,267,251,77,217,283,301,196,768,411,212,82,431,571,220,201,296,196,219,335,89,250,213,71,299,578,227,605,16,624,273,269,345,311,289,20,221,310,225,615,363,223,578,227,502,459,605,393,243,432,880,83,311,920,459,245,19,250,66,310,201,308,910,198,250,706,246,320,212,971,378,84,221,250,706],"3":[246,199,224,279,83,390,487,957,289,19,459,428,24,212,77,311,670,459,916,221,310,908,225,615,363,796,511,629,459,289,511,289,393,71,311,245,511,502,459,493,511,670,250,66,310,209,317,783,548,212,82,431,571,738,201,250,706,246,320,490,196,232,364,377,316,559,234,851,400,543,230,431,203,458,221,371,975,227,267,859,201,221,196,344,67,382,994,253,67,68,89,83,215,311,230,235,76,84,223,310,209,795,977,205,438,260,951,194,273,216,196,544,274,407,426,71,250,706,246,320,201,445,671,202,226,216,227,263],"4":[298,253,292,89,245,459,289,818,274,768,71,199,326,273,637,209,252,261,455,224,899,218,224,350,73,343,335,83,301,454,77,204,296,196,768,411,201,281,89,77,77,269,82,452,224,899,218,472,655,83,209,261,768,71,199,496,301,198,239,212,82,283,294,239,201,225,499,159,590,204,254,335,306,216,788,274,212,82,283,72,223,577,89,440,198,548,301,198,239,212,286,280,239,201,400,499,392,217,80,221,78,199,159,204,71,216,201,225,301,788,274,232,636,223,371,194,383,223,198,577,89,209,397,83,957,201,198,409],"5":[678,472,655,301,198,212,82,283,294,201,225,198,358,420,301,198,212,286,280,209,252,261,377,316,559,234,851,301,267,757,417,841,467,485,402,69,201,296,405,279,83,295,767,222,216,298,201,225,407,416,284,993,284,209,261,237,68,713,418,741,375,73,380,296,250,706,246,320,669,528,383,83,550,212,680,223,209,795,205,438,260,83,527,848,201,221,250,822,201,198,237,68,224,278,408,196,246,65,88,220,299,197,301,219,549,227,196,344,563,197,981,88,201,445,198,981,88,301,219,228,75,213,578,307,198,331,208,218],"6":[212,680,223,201,734,226,223,198,237,68,224,278,408,209,252,261,472,316,328,691,941,218,282,14,267,251,77,217,283,301,198,764,220,250,706,246,199,201,282,249,217,283,196,760,220,283,209,261,506,753,346,390,723,89,203,273,243,217,201,225,684,293,212,973,204,982,778,73,417,201,308,910,232,89,66,82,271,83,390,434,76,965,539,227,205,438,260,221,198,206,627,203,856,444,358,568,216,804,410,547,76,422,209,261,506,753,346,684,293,231,333,223,85,736,307,196,845,218,878,280,333,790,466,252,261,494,246,82],"7":[327,218,282,14,196,760,220,283,293,819,496,371,604,405,828,263,198,423,207,320,446,201,400,390,250,394,223,221,282,14,267,251,77,217,283,209,252,261,405,828,263,198,472,655,83,218,282,14,196,760,220,283,390,237,68,371,237,68,277,194,73,354,204,201,732,244,607,218,282,14,267,251,77,217,283,390,305,560,371,305,560,277,194,73,354,204,209,252,261,423,207,320,446,218,198,472,655,218,282,14,196,760,220,283,301,371,220,411,371,237,68,201,732,295,218,282,14,267,251,77,217,283,301,942,251,89,305,560],"8":[371,723,89,224,842,237,68,209,252,238,238,317,822,212,89,67,234,238,238,252,304,303,842,282,14,267,251,77,217,283,237,532,359,88,779,230,208,260,379,646,543,490,267,228,574,227,196,212,217,422,571,250,213,71,299,218,672,16,459,672,21,955,273,269,345,311,428,511,245,459,428,511,428,221,310,201,756,226,230,222,216,230,208,447,319,196,203,76,420,264,947,366,199,203,529,69,209,259,761,194,998,229,417,205,438,260,83,221,203,327,495,708,196,489,242,264,230,235,76,744,214,303,842,201,305,607,598,800],"9":[301,817,717,749,678,201,225,196,232,364,277,598,800,204,230,842,209,261,214,303,842,212,217,337,216,198,253,71,71,83,274,578,227,837,904,299,83,201,897,483,223,263,198,194,303,473,208,447,201,569,257,458,227,468,224,234,343,335,83,209,304,303,222,216,212,217,488,223,253,71,71,83,390,203,810,227,293,239,219,199,337,204,239,225,684,293,824,718,462,198,544,209,252,261,253,71,71,83,232,208,309,319,254,420,201,225,198,768,86,65,69,203,87,273,227,198,206,475,833,70,571,756,543,231,337,678,296,198],"10":[205,298,220,212,260,592,745,201,577,89,223,263,159,90,79,343,76,858,851,209,795,285,491,896,270,86,216,697,230,235,76,745,225,250,404,83,274,916,459,428,21,792,83,209,929,198,244,825,230,235,76,84,201,198,536,85,86,213,515,194,368,216,263,196,740,472,316,199,227,198,486,512,201,225,486,343,745,196,219,242,72,229,250,347,328,89,234,209,261,536,85,86,213,243,216,390,765,202,264,359,213,221,198,206,627,201,225,390,634,210,264,876,201,308,910,543,390,876,227,293,212,422,614,218,231,278,71,223],"11":[972,579,388,219,260,859,83,209,530,301,928,273,380,295,669,245,768,86,65,221,253,292,89,464,562,802,833,86,272,216,227,198,219,242,72,229,224,72,626,209,312,787,543,237,532,196,212,217,422,571,250,213,71,299,218,916,230,77,311,796,511,502,25,221,310,201,198,536,85,86,213,243,216,409,429,444,219,260,859,83,225,285,367,444,486,512,250,272,216,209,252,238,238,300,333,608,286,233,238,238,252,282,249,217,283,267,251,77,217,283,301,824,772,973,198,254,663,277,253,226,612,775,76,373,229,339,298,220],"12":[340,254,349,294,78,318,210,683,227,198,240,90,79,345,225,259,210,375,67,79,201,410,867,198,265,222,84,229,742,65,209,530,301,471,846,242,221,659,218,198,259,204,211,199,476,69,220,742,65,201,669,230,638,223,340,198,203,395,233,253,404,218,247,202,563,201,225,308,362,669,198,254,663,277,206,328,767,404,218,198,265,76,394,742,65,209,261,254,349,294,78,77,419,883,306,524,390,824,221,198,318,210,591,71,406,214,74,412,83,236,89,83,70,74,412,213,225,318,412,70,636,65,201,221,83,446,198,704],"13":[321,229,247,275,67,234,209,252,261,753,346,684,293,231,272,271,204,587,784,267,213,269,229,417,231,333,197,321,883,306,524,201,496,206,271,216,80,202,256,883,306,291,201,225,697,400,490,231,829,71,204,231,467,227,947,366,253,465,395,388,883,306,291,203,529,216,201,729,83,551,264,231,467,227,486,978,291,227,198,844,222,554,86,275,200,408,209,261,455,218,198,399,301,198,883,306,291,218,250,706,246,320,340,254,349,294,78,318,210,683,201,400,490,504,237,652,401,227,281,198,239,230,271,78,420,277,203,302],"14":[250,706,246,199,239,209,261,883,306,524,221,198,259,204,211,199,476,69,220,742,65,390,231,333,197,321,340,244,607,221,198,775,76,373,229,339,298,220,209,261,250,404,231,333,197,321,883,306,291,301,824,221,198,318,269,294,508,83,466,203,251,520,216,340,198,339,419,320,67,195,352,69,360,231,333,197,321,340,244,607,713,791,204,221,198,318,663,742,65,371,811,76,424,425,786,248,209,252,775,84,303,381,83,490,504,746,227,450,228,501,298,282,14,267,251,77,217,283,227,752,903,69,222,325,201,308,362,83],"15":[446,548,338,85,635,69,220,753,346,781,281,198,159,204,551,234,212,314,66,201,247,220,67,199,224,330,260,283,209,265,269,687,323,16,20,225,323,17,20,201,496,955,233,250,706,246,199,768,86,65,69,360,734,620,340,232,208,67,294,346,221,300,302,204,197,201,445,198,753,346,992,410,527,479,928,372,76,736,817,209,252,238,238,338,67,727,89,238,238,252,240,68,512,282,14,267,251,77,217,283,250,388,263,198,963,242,222,598,76,70,319,897,299,83,218,796,459,916,16,922,345,311,796,459,493,25,18,214]}}
"""

train_df = pd.read_json(train_data)
validation_df = pd.read_json(validation_data)

ray_train = ray.data.from_pandas(train_df)
ray_validation = ray.data.from_pandas(validation_df)
print(ray_train)
print(ray_validation)


# from datasets import Dataset
# import ray

# lm_dataset_train = Dataset.from_parquet('../data/train/*.parquet', cache_dir='/tmp/')

# ray_train_ds = ray.data.from_huggingface(
#     lm_dataset_train
# )

2023-03-15 05:28:25,011	INFO worker.py:1553 -- Started a local Ray instance.


Dataset(num_blocks=1, num_rows=16, schema={input_ids: object, attention_mask: object, labels: object})
Dataset(num_blocks=1, num_rows=16, schema={input_ids: object, attention_mask: object, labels: object})


TypeError: `dataset` must be a `datasets.Dataset` or `datasets.DatasetDict`, got <class 'str'>

# Train model and save a checkpoint

In [21]:
model_checkpoint = 'bigscience/bloomz-560m'

def train_function(train_dataset, eval_dataset=None, **config):
    
    model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
    evaluation_strategy = (
        config.pop("evaluation_strategy", "epoch") if eval_dataset else "no"
    )
    training_args = TrainingArguments(
        f"{model_checkpoint}-amazon-customer-reviews-bloomz-560m",
#        evaluation_strategy=evaluation_strategy,
#        logging_strategy=config.pop("logging_strategy", "epoch"),
        max_steps=10,
        num_train_epochs=config.pop("epochs", 1),
        learning_rate=config.pop("learning_rate", 2e-5),
#        weight_decay=0.01,
        disable_tqdm=True,
        no_cuda=True,
        save_strategy=config.pop("save_strategy", "no"),
        **config,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )
    return trainer

In [28]:
scaling_config = ScalingConfig(num_workers=num_cpus-20, 
                               use_gpu=False,
#                               _max_cpu_fraction_per_node = 0.8    
)

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=train_function,
    trainer_init_config={"epochs": 1, "save_strategy": "no"},
    scaling_config=scaling_config,
    datasets={"train": ray_train,
              #"evaluation": ray_validation
             },
)

result = trainer.fit()

print(result.checkpoint)
print(result.metrics)

/opt/conda/lib/python3.7/site-packages/ray/train/base_trainer.py:361: UserWarning: Executing `.fit()` may leave less than 20% of CPUs in this cluster for Dataset execution, which can lead to resource contention or hangs. To avoid this, reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer scaling_config. See https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune for more info.
  trainable=trainable, param_space=param_space, run_config=self.run_config
2023-03-15 05:28:58,238	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(RayTrainWorker pid=49924) 2023-03-15 05:29:08,985	INFO config.py:87 -- Setting up process group for: env:// [rank=0, world_size=76]
(HuggingFaceTrainer pid=49655) 2023-03-15 05:29:10,989	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[randomize_block_order]
(HuggingFaceTrainer pid=49655) /opt/conda/lib/python3.7/site-packages/ray/data/_internal/bulk_dataset_iterator.py:109: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterator.html for full DatasetIterator docs.
(HuggingFaceTrainer pid=49655)   "session.get_dataset_shard returns a ray.data.DatasetIterator "
(RayTrainWorker pid=49930) /opt/conda/lib/python3.7/site-packages/ray/data/_internal/bulk_dataset_iterator.py:109: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a

Trial name,date,experiment_id,hostname,node_ip,pid,timestamp,trial_id
HuggingFaceTrainer_49150_00000,2023-03-15_05-29-01,9a43b3801df94a66be2a984daa028726,datascience-1-0-ml-m5-24xlarge-d807f334101db4b858a3c6bf1fdd,169.255.255.2,49655,1678858141,49150_00000


2023-03-15 05:29:36,529	ERROR tune.py:794 -- Trials did not complete: [HuggingFaceTrainer_49150_00000]
2023-03-15 05:29:36,530	INFO tune.py:799 -- Total run time: 38.30 seconds (38.22 seconds for the tuning loop).


RayTaskError(ZeroDivisionError): [36mray::_Inner.train()[39m (pid=49655, ip=169.255.255.2, repr=HuggingFaceTrainer)
  File "/opt/conda/lib/python3.7/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/opt/conda/lib/python3.7/site-packages/ray/train/_internal/utils.py", line 54, in check_for_failure
    ray.get(object_ref)
ray.exceptions.RayTaskError(ZeroDivisionError): [36mray::RayTrainWorker._RayTrainWorker__execute()[39m (pid=49933, ip=169.255.255.2, repr=<ray.train._internal.worker_group.RayTrainWorker object at 0x7fd8ae02e050>)
  File "/opt/conda/lib/python3.7/site-packages/ray/train/_internal/worker_group.py", line 31, in __execute
    raise skipped from exception_cause(skipped)
  File "/opt/conda/lib/python3.7/site-packages/ray/train/_internal/utils.py", line 129, in discard_return_wrapper
    train_func(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/ray/train/huggingface/huggingface_trai

# Predict from checkpoint

In [ ]:
prompts = pd.DataFrame(
    ["Complete me", "And me", "Please complete"], columns=["sentences"]
)

In [ ]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.huggingface import (
    HuggingFacePredictor,
)

model_checkpoint = "bigscience/bloomz-560m"

predictor = BatchPredictor.from_checkpoint(
    result.checkpoint,
    HuggingFacePredictor,
    task="text-generation",
    tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)
)

predictions = predictor.predict(ray.data.from_pandas(prompts))
print(predictions)
assert predictions.count() == 3

# Train with an evaluation metric such as "glue"

In [ ]:
from datasets import load_metric

metric = load_metric("glue", "cola")

def train_function_with_metric(train_dataset, eval_dataset=None, **config):
    print(metric)
    return train_function(train_dataset, eval_dataset=eval_dataset, **config)

scaling_config = ScalingConfig(num_workers=num_cpus-2, use_gpu=False)

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=train_function_with_metric,
    trainer_init_config={"epochs": 1},
    scaling_config=scaling_config,
    datasets={"train": ray_train, "evaluation": ray_validation},
)
trainer.fit()


# Resume training from a checkpoint using more data

In [ ]:
# trainer2 = HuggingFaceTrainer(
#     trainer_init_per_worker=train_function,
#     trainer_init_config={
#         "epochs": 1,
#         "save_strategy": "epoch",
#     },  # this will train for 1 epoch: 5 - 4 = 1
#     scaling_config=scaling_config,
#     datasets={"train": ray_train, "evaluation": ray_validation},
#     resume_from_checkpoint=result.checkpoint,
# )
# result2 = trainer2.fit()

# assert result2.metrics["epoch"] == 1
# assert result2.metrics["training_iteration"] == 1
# assert result2.checkpoint
# assert isinstance(result2.checkpoint, HuggingFaceCheckpoint)
# assert "eval_loss" in result2.metrics

In [ ]:
# predictor = BatchPredictor.from_checkpoint(
#     result2.checkpoint,
#     HuggingFacePredictor,
#     task="text-generation",
#     tokenizer=AutoTokenizer.from_pretrained(model_checkpoint),
# )

# predictions = predictor.predict(ray.data.from_pandas(prompts))
# assert predictions.count() == 3

# Test hyper-parameter tuner

In [ ]:
# Tests if checkpointing and restoring during tuning works correctly.
from ray import tune
from ray.tune import Tuner
from ray.tune.schedulers.async_hyperband import ASHAScheduler
from ray.tune.schedulers.resource_changing_scheduler import (
    DistributeResources,
    ResourceChangingScheduler,
)

scaling_config = ScalingConfig(
    num_workers=num_cpus-2, 
    use_gpu=False, 
    trainer_resources={"CPU": 0},
#    _max_cpu_fraction_per_node = 0.8    
)

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=train_function,
    scaling_config=scaling_config,
    datasets={"train": ray_train, "evaluation": ray_validation},
)

tune_epochs = 1
tuner = Tuner(
    trainer,
    param_space={
        "trainer_init_config": {
            "learning_rate": tune.loguniform(2e-6, 2e-5),
            "epochs": tune_epochs,
            "save_strategy": "epoch",
        }
    },
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
        num_samples=3,
        scheduler=ResourceChangingScheduler(
            ASHAScheduler(
                max_t=tune_epochs,
            ),
            resources_allocation_function=DistributeResources(
                add_bundles=True, reserve_resources={"CPU": 1}
            ),
        ),
    ),
)
tuner_results = tuner.fit()
print(tuner_results)